In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import mysql.connector
import pandas as pd
import json

In [2]:
def open_database():
    con = mysql.connector.connect(user='dbadmin', password='washywashy',host='127.0.0.1',database='tests')
    return con, con.cursor()

def read_csv(filename):
    df = pd.read_csv(filename, low_memory=False)
    return df

def select_columns(df, columns):
    return df[columns]

def get_data():
    df = read_csv('./data/movies_imbd.csv')
    return df

def get_column_df(col):
    df = read_csv('./data/movies_imbd.csv')
    df = select_columns(df,[col])
    df = df[~pd.isnull(df[col])]
    df.fillna("",inplace=True)
    return df

In [3]:
def create_imbd_movies_table():
    con, cursor = open_database()
    cursor.execute("drop table if exists tests.imbd_movies")
    create_table_sql = """
    create table tests.imbd_movies (
            id int,
            imdb_id varchar(150),
            original_title varchar(250),
            title varchar(250),
            original_language varchar(120),
            overview varchar(1000),
            tagline varchar(560),
            release_date varchar(120),
            status varchar(50),
            budget varchar(1000),
            revenue float,
            runtime float,
            adult varchar(20),
            popularity float,
            vote_average float,
            vote_count float,
            homepage varchar(550),
            poster_path varchar(550),
            video varchar(20)

    )
    """
    cursor.execute(create_table_sql)
    con.commit()    
    con.close()

def insert_into_imbd_movies_table():
    df = read_csv('./data/movies_imbd.csv')
    df = select_columns(df, ['id','imdb_id','original_title', 'title','original_language','overview','tagline', 'release_date', 'status', 'budget', 'revenue', 'runtime', 'adult', 'popularity', 'vote_average', 'vote_count','homepage', 'poster_path', 'video'])
    df = df.fillna(0)

    values = []
    i = 1
    for x in df.values:
        data = tuple(x)
        i += 1
        values.append(data)
        con, cursor = open_database()
        try:
            insert_sql = "insert into tests.imbd_movies (id,imdb_id,original_title, title,original_language,overview,tagline, release_date, status, budget, revenue, runtime, adult, popularity, vote_average, vote_count,homepage, poster_path, video) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execut(insert_sql, data)
            print('worked', i, x)
        except:
            print("--->", i, x)
            
#     data =  tuple([tuple(x) for x in df.values])
   
    con.commit()   
    con.close()    

In [4]:
create_imbd_movies_table()
# insert_into_imbd_movies_table()

In [43]:
float("10")

10.0

In [42]:
def clean_id(idn, my_lambda):
    try:
        idn = my_lambda(idn)
        return idn
    except:
        return my_lambda(0)

df = read_csv('./data/movies_imbd.csv')
df = select_columns(df, ['id','imdb_id','original_title', 'title','original_language','overview','tagline', 'release_date', 'status', 'budget', 'revenue', 'runtime', 'adult', 'popularity', 'vote_average', 'vote_count','homepage', 'poster_path', 'video'])
df = df.fillna(0)

df['id'] = df.apply(lambda x: clean_id(x['id'], lambda a : int(a)), axis=1)
df['revenue'] = df.apply(lambda x: clean_id(x['revenue'], lambda a : float(a)), axis=1)
df['runtime'] = df.apply(lambda x: clean_id(x['runtime'], lambda a : float(a)), axis=1)
df['popularity'] = df.apply(lambda x: clean_id(x['popularity'], lambda a : float(a)), axis=1)
df['vote_average'] = df.apply(lambda x: clean_id(x['vote_average'], lambda a : float(a)), axis=1)
df['vote_count'] = df.apply(lambda x: clean_id(x['vote_count'], lambda a : float(a)), axis=1)
df['adult'] = df.apply(lambda x: ('True' if x.adult == 'True' else 'False'), axis=1)
df = df.fillna('')

In [44]:
def get_sql_field(number_of_fields):
    all_fields = ['id','imdb_id','original_title','title','original_language','overview','tagline', 'release_date', 'status', 'budget', 'revenue', 'runtime', 'adult', 'popularity', 'vote_average', 'vote_count','homepage', 'poster_path', 'video']
    number_of_fields = number_of_fields if number_of_fields > 0 else len(all_fields)
    fields = all_fields[0:number_of_fields]
    sql = "insert into tests.imbd_movies (" + ",".join(fields)+ ") values (" + ",".join(["%s" for i in range(len(fields))]) +")"
    return sql, fields
    
print(get_sql_field(3))

('insert into tests.imbd_movies (id,imdb_id,original_title) values (%s,%s,%s)', ['id', 'imdb_id', 'original_title'])


In [45]:
con, cursor = open_database()
con.autocommit = True
cursor.execute('delete from tests.imbd_movies')

sql, fields =  get_sql_field(0)
data = []
for x in df[fields].values:
    data.append(tuple(x))
    

cursor.executemany(sql, data)


con.close()    